## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [3]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("Vacation_Search/WeatherPy_vacation.csv")
vacation_df.tail(20)

,City_ID,City,Country,Max Temp,Weather description,Lat,Lng,Hotel Name
333,664,Itarema,BR,78.30,clear sky,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
334,665,Santa Rosa,PH,86.11,light rain,14.3122,121.1114,Erlyn’s Sta Rosa City Home
335,666,Black River,JM,80.74,clear sky,18.0264,-77.8487,Idlers' Rest Beach Hotel
336,671,Kapoeta,SS,77.95,broken clouds,4.7717,33.5903,Eastgate Hotel
337,673,Honghu,CN,87.33,broken clouds,29.8206,113.4621,文泉花园酒店
338,674,Port Blair,IN,82.44,mist,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
339,676,Kindu,CD,70.12,broken clouds,-2.9500,25.9500,Résidence Okanda
340,680,Wuchang,CN,72.39,overcast clouds,44.9143,127.1500,杏花山
341,681,Goderich,CA,77.32,overcast clouds,43.7501,-81.7165,Harmony Inn
342,682,West Richland,US,88.05,clear sky,46.3043,-119.3614,Horn Rapids RV Resort


In [5]:
# Refine your search with different weather criteria to get cities that are close together.
preferred_cities_df = vacation_df.loc[vacation_df['Country'] == 'JP']
preferred_cities_df.head(20)

,City_ID,City,Country,Max Temp,Weather description,Lat,Lng,Hotel Name
14,37,Shimoda,JP,84.56,overcast clouds,34.6667,138.9500,Shimoda Tokyu Hotel
28,72,Toba,JP,80.60,overcast clouds,34.4833,136.8500,ホテル アルティア鳥羽
38,94,Hasaki,JP,82.80,moderate rain,35.7333,140.8333,Inubosaki Kanko Hotel
76,187,Nishihara,JP,86.05,overcast clouds,26.1842,127.7558,Minshuku Agaihama
97,230,Iwaki,JP,89.02,overcast clouds,37.0500,140.8833,Toyoko Inn Iwaki Ekimae
109,256,Katsuura,JP,81.32,overcast clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
173,379,Hirara,JP,89.46,broken clouds,24.8000,125.2833,Hotel Atoll Emerald Miyakojima
199,438,Misawa,JP,81.61,scattered clouds,40.6836,141.3597,Hoshino Resorts Aomoriya
235,507,Itoman,JP,86.77,broken clouds,26.1247,127.6694,Southern Beach Hotel & Resort Okinawa
259,541,Nanae,JP,70.03,broken clouds,41.8833,140.6833,とみや旅館


In [112]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_content = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather description</dt><dd>{Weather description} and {Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_content.format(**row) for index, row in preferred_cities_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = preferred_cities_df[["Lat", "Lng"]]

In [113]:
# 4a. Add a marker layer for each city to the map.
locations = preferred_cities_df[["Lat", "Lng"]]
max_temp = preferred_cities_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [59]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df["City"] == "Shimoda"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Shimoda"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Katsuura"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Hasaki"] 
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Iwaki"] 

In [62]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start[["Lat", "Lng"]].to_numpy()[0]
end = vacation_end[["Lat", "Lng"]].to_numpy()[0]
stop1 = vacation_stop1[["Lat", "Lng"]].to_numpy()[0]
stop2 = vacation_stop2[["Lat", "Lng"]].to_numpy()[0]
stop3 = vacation_stop3[["Lat", "Lng"]].to_numpy()[0]

In [63]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
JP_trip = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2, stop3],
        travel_mode='DRIVING')
fig.add_layer(JP_trip)
fig

Figure(layout=FigureLayout(height='420px'))

In [64]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Weather description,Lat,Lng,Hotel Name
0,37,Shimoda,JP,84.56,overcast clouds,34.6667,138.9500,Shimoda Tokyu Hotel
1,256,Katsuura,JP,81.32,overcast clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
2,94,Hasaki,JP,82.80,moderate rain,35.7333,140.8333,Inubosaki Kanko Hotel
3,230,Iwaki,JP,89.02,overcast clouds,37.0500,140.8833,Toyoko Inn Iwaki Ekimae


In [75]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_content = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather description</dt><dd>{Weather description} and {Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_content.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [76]:
# 11a. Add a marker layer for each city to the map.
locations = itinerary_df[["Lat", "Lng"]]
max_temp = itinerary_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))